# Imports 

In [1]:
import matplotlib.patches as mpatches
from matplotlib.patches import Rectangle
from matplotlib.ticker import PercentFormatter

# Data

In [2]:
stanton = pd.read_csv('../data/giancarlo-stanton.csv')
stanton.drop(columns = ['Unnamed: 0'], inplace = True)

# Font Dictionary
font_title = {
    'size': 12,
    'weight': 'bold',
    'verticalalignment': 'center_baseline',
    'horizontalalignment': 'center'
}

pd.set_option('max_columns', None)
print(stanton.shape)
stanton.head(2)

(2426, 89)


,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,fielder_2,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,pitcher.1,fielder_2.1,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,is_strike,pitch_count,plate_-x,pfx_-x,swing_miss
0,FF,2021-10-03,95.9,-1.78,6.35,"Stanton, Giancarlo",519317,608379,out,hit_into_play,7.0,Giancarlo Stanton flies out to right fielder R...,R,R,R,NYY,TB,X,9.0,fly_ball,2,2,2021,-0.89,14.16,-0.69,2.20,NaN,NaN,650402.0,2,4,Bot,181.19,120.39,572287,4.936664,-139.502697,-8.110468,-12.687337,30.485721,-15.110574,3.66,1.68,278.0,85.4,28.0,96.8,2024.0,6.8,632239,608379,572287,596847,664040,621563,677551,640457,595281,668227,53.73,0.071,0.086,0.0,1.0,0.0,0.0,3.0,30,6,4-Seam Fastball,0,0,0,0,0,0,0,0,Standard,Standard,210.0,-0.028,-0.192,1,2-2,0.69,10.68,0
1,FF,2021-10-03,96.1,-1.72,6.31,"Stanton, Giancarlo",519317,608379,NaN,ball,11.0,Giancarlo Stanton flies out to right fielder R...,R,R,R,NYY,TB,B,NaN,NaN,1,2,2021,-0.61,18.12,-0.51,4.88,NaN,NaN,650402.0,2,4,Bot,NaN,NaN,572287,4.630125,-139.866434,-1.607320,-9.063359,31.957645,-12.240695,3.80,1.90,NaN,NaN,NaN,96.8,2238.0,6.7,632239,608379,572287,596847,664040,621563,677551,640457,595281,668227,53.81,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30,5,4-Seam Fastball,0,0,0,0,0,0,0,0,Standard,Standard,209.0,0.000,0.037,0,1-2,0.51,7.32,0
